In [17]:
import json

import os

import pandas as pd
import numpy as np

In [18]:
LANG_CODES_TO_NAMES = {
    "en": "English",
    "ru": "Russian",
    "ko": "Korean",
    "cmn-Hant": "Traditional Chinese",
    "ja": "Japanese",
    "de": "German",
    "es": "Spanish",
}


In [19]:
# Function to get the script directory
def get_script_dir():
    """Returns the directory where the script is located."""
    dir = os.getcwd()
    if dir.endswith("modTiers"):
        return dir[:-len("modTiers")]
    return dir

# Function to generate file path
def get_file_path(base_dir: str, file: str, lang: str, is_en=False):
    return f"{base_dir}{lang if not is_en else 'en'}/{file}.json"

# Function to load a JSON file
def load_file(base_dir: str, file: str, lang: str, is_en=False):
    path = get_file_path(base_dir, file, lang, is_en)
    with open(path, encoding="utf-8") as f:
        return json.load(f)

# Function to load a DataFrame
def load_df(base_dir: str, file: str, lang: str, is_en=False):
    path = get_file_path(base_dir, file, lang, is_en)
    return pd.read_json(path, encoding="utf-8")

def from_json(base_dir: str, file: str, lang: str, is_en=False):
    file = load_file(base_dir, file, lang, is_en)
    return pd.DataFrame(file)

In [20]:
cwd = get_script_dir()
base_dir = cwd + "/tables/"
lang = "en"

In [21]:
mods = from_json(base_dir, "Mods", lang).drop(columns=["_index"])
base_items = from_json(base_dir, "BaseItemTypes", lang).drop(columns=["_index"])
gold_mod_prices = from_json(base_dir, "GoldModPrices", lang).drop(columns=["_index"])
tags = from_json(base_dir, "Tags", lang).drop(columns=["_index"])

# Apply filter stuff to tags via spawn weight

In [22]:
gold_mod_prices

,Mod,Tags,SpawnWeight
0,0,"[2, 3, 26, 38, 41, 42, 44, 11, 15, 12, 1010, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
1,1,"[2, 3, 26, 38, 41, 42, 44, 11, 15, 12, 1010, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
2,2,"[2, 3, 26, 38, 41, 42, 44, 11, 15, 12, 1010, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
3,3,"[2, 3, 26, 38, 41, 42, 44, 11, 15, 12, 1010, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
4,4,"[2, 3, 26, 38, 41, 42, 44, 11, 15, 12, 1010, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
...,...,...,...
1602,3607,[],[]
1603,3608,[],[]
1604,3609,[],[]
1605,3610,[],[]


In [23]:
replace = {
    38: 7,
    39: 7,
    40: 7,
    41: 7,
    42: 7,
    43: 7,
    44: 7,
    226: 1,
    227: 1,
    228: 1,
    229: 1,
    230: 1,
}
def process_tags(tags_list, weights_list):
    # Replace tag IDs
    new_tags = [replace[t] if t in replace else t for t in tags_list]
    
    # Remove sequential duplicate 7s along with corresponding SpawnWeights
    filtered_tags, filtered_weights = [], []
    prev_tag = None
    for tag, weight in zip(new_tags, weights_list):
        if (tag == 7 and prev_tag == 7) or (tag == 1 and prev_tag == 1):
            continue  # Skip duplicate sequential 7
        filtered_tags.append(tag)
        filtered_weights.append(weight)
        prev_tag = tag

    return filtered_tags, filtered_weights

# Apply transformation to each row
gold_mod_prices["ProcessedTags"], gold_mod_prices["ProcessedSpawnWeight"] = zip(
    *gold_mod_prices.apply(lambda row: process_tags(row["Tags"], row["SpawnWeight"]), axis=1)
)

In [24]:
gold_mod_prices.iloc[486]

Mod                                  486
Tags                    [1012, 5, 81, 0]
SpawnWeight                 [1, 1, 1, 0]
ProcessedTags           [1012, 5, 81, 0]
ProcessedSpawnWeight        [1, 1, 1, 0]
Name: 486, dtype: object

## category expansion stuff

In [25]:
category_expansion = {
    7: {16, 4, 22, 25, 1, 45},  # ARMOUR -> Armour pieces
    8: {5,12,11,15,1010,14,13,1014,1012,566}
}

# Function to apply logical filtering based on category expansion and removal of 0-weighted tags
def filter_tags_and_remove_category(tags_list, weights_list, category_expansion):
    tag_dict = {tag: bool(weight) for tag, weight in zip(tags_list, weights_list)}

    # Process category expansions
    expanded_tags = set()
    for tag in tags_list:
        if tag in category_expansion:
            for expanded_tag in category_expansion[tag]:
                if expanded_tag not in tag_dict:
                    tag_dict[expanded_tag] = True  # Mark new expanded tags as 1 (true)
            expanded_tags.add(tag)  # Mark category for removal

    # Remove tags with weight 0 and also remove the original category tags
    filtered_tags = [tag for tag, keep in tag_dict.items() if keep and tag not in expanded_tags]

    return filtered_tags

# Apply the filtering to each row
gold_mod_prices["FinalTags"] = gold_mod_prices.apply(
    lambda row: filter_tags_and_remove_category(row["ProcessedTags"], row["ProcessedSpawnWeight"], category_expansion),
    axis=1
)

In [26]:
gold_mod_prices.iloc[760]

Mod                     760
Tags                    [0]
SpawnWeight             [0]
ProcessedTags           [0]
ProcessedSpawnWeight    [0]
FinalTags                []
Name: 760, dtype: object

In [27]:
gold_explode = gold_mod_prices.explode("FinalTags")
merged_gold_explode = gold_explode.merge(tags, left_on="FinalTags", right_index=True, how="left")
fixed_gold_mod_prices = merged_gold_explode.groupby("Mod").agg({
    "Id": list,  # Keep the Tag IDs as lists
}).reset_index().rename(columns={"Id": "Tags"})
# fixed_gold_mod_prices["SpawnWeight"] = gold_mod_prices["ProcessedSpawnWeight"]
# fixed_gold_mod_prices.iloc[760]
fixed_gold_mod_prices

,Mod,Tags
0,0,"[ring, amulet, belt, mace, axe, sword, spear, ..."
1,1,"[ring, amulet, belt, mace, axe, sword, spear, ..."
2,2,"[ring, amulet, belt, mace, axe, sword, spear, ..."
3,3,"[ring, amulet, belt, mace, axe, sword, spear, ..."
4,4,"[ring, amulet, belt, mace, axe, sword, spear, ..."
...,...,...
1602,3607,[nan]
1603,3608,[nan]
1604,3609,[nan]
1605,3610,[nan]


In [28]:
fixed_gold_mod_prices.iloc[760]

Mod       760
Tags    [nan]
Name: 760, dtype: object

In [29]:
gold_with_readable_tags_df = pd.merge(
    fixed_gold_mod_prices[
    fixed_gold_mod_prices["Tags"].apply(
        lambda x: not (isinstance(x, list) and len(x) == 1 and isinstance(x[0], float) and np.isnan(x[0]))
    )
],
    mods,
    left_on="Mod",
    right_index=True,
    how="left"
)[["Id", "Tags", "GenerationType"]]

gold_with_readable_tags_df

,Id,Tags,GenerationType
0,Strength1,"[ring, amulet, belt, mace, axe, sword, spear, ...",2
1,Strength2,"[ring, amulet, belt, mace, axe, sword, spear, ...",2
2,Strength3,"[ring, amulet, belt, mace, axe, sword, spear, ...",2
3,Strength4,"[ring, amulet, belt, mace, axe, sword, spear, ...",2
4,Strength5,"[ring, amulet, belt, mace, axe, sword, spear, ...",2
...,...,...,...
1351,MinionLife2,[sceptre],2
1352,MinionLife3,[sceptre],2
1353,MinionLife4,[sceptre],2
1354,MinionLife5,[sceptre],2


In [30]:
GENERATION_TYPE = {
    1: "prefix",
    2: "suffix",
    3: "intrinsic",
    4: "nemesis",
    5: "corrupted",
    6: "bloodlines",
    7: "torment",
    8: "tempest",
    9: "talisman",
    10: "enchantment",
    11: "essence",
    12: "unused",
    13: "bestiary",
    14: "delve",
    15: "synthesis unknown",
    16: "synthesis globals",
    17: "synthesis bonus",
    18: "blight",
    19: "blight tower",
    20: "monster affliction",
    21: "enkindling orb",
    22: "instilling orb",
    23: "expedition logbook",
    24: "scourge benefit",
    25: "scourge detriment",
    26: "scourge gimmick",
    27: "unused",
    28: "searing exarch",
    29: "eater of worlds",
    30: "archnemesis",
    31: "crucible passive skill tree",
    32: "crucible passive skill tree mutation",
    33: "affliction wisps",
    34: "necropolis downside",
    35: "necropolis upside"
}

gold_with_readable_tags_df["Generation"] = gold_with_readable_tags_df["GenerationType"].map(GENERATION_TYPE)
generation_mapped_df = gold_with_readable_tags_df[["Id", "Tags", "Generation"]]

In [31]:
gold_with_readable_tags = {x["Id"]:x["Tags"] for x in generation_mapped_df.to_dict(orient="records")}
